In [1]:
!git clone https://github.com/jantic/DeOldify.git

fatal: destination path 'DeOldify' already exists and is not an empty directory.


In [ ]:
cd DeOldify

In [ ]:
!pip install -r requirements-colab.txt

In [4]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

<DeviceId.GPU0: 0>

In [5]:
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from deoldify.generators import *
from deoldify.critics import *
from deoldify.dataset import *
from deoldify.loss import *
from deoldify.save import *

NumExpr defaulting to 12 threads.


In [6]:
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

C:\Users\patri\OneDrive\Desktop\final_masters\DeOldify\deoldify\visualize.py:223: SyntaxWarning: invalid escape sequence '\.'
  if re.search('.*?\.jpg', f):


In [ ]:
import kagglehub
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from PIL import Image, ImageDraw

general = kagglehub.dataset_download("akash2sharma/tiny-imagenet")
medical = kagglehub.dataset_download("tawsifurrahman/covid19-radiography-database")
astronomy = kagglehub.dataset_download("razaimam45/spacenet-an-optimally-distributed-astronomy-data")
cctv = kagglehub.dataset_download("aryashah2k/large-scale-multicamera-detection-dataset")

train_ds = './data/custom_dataset/norm/full_train/'
sample_train = './data/custom_dataset/norm/train/'
test_ds = './data/custom_dataset/norm/test/'
train_bandw = './data/custom_dataset/bandw/train/'

os.makedirs(train_ds, exist_ok=True)
os.makedirs(sample_train, exist_ok=True)
os.makedirs(test_ds, exist_ok=True)
os.makedirs(train_bandw, exist_ok=True)

In [8]:
def convert_and_copy_images(source_dir, target_dir, max_images=10000):
    source_dir = Path(source_dir)
    target_dir = Path(target_dir)
    target_dir.mkdir(parents=True, exist_ok=True)

    # Start counting from how many files already exist
    existing_files = list(target_dir.glob('*.jpg'))
    image_count = len(existing_files)

    allowed_exts = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.dcm']
    all_files = list(source_dir.rglob("*"))

    copied = 0  # how many images we copy *this time*

    for f in tqdm(all_files, desc=f"Processing {source_dir.name}"):
        ext = f.suffix.lower()
        if ext not in allowed_exts:
            continue

        if copied >= max_images:
            break

        try:
            out_path = target_dir / f"{image_count:05d}.jpg"

            if ext == '.dcm':
                dcm = pydicom.dcmread(f)
                pixel_array = dcm.pixel_array.astype(np.float32)
                pixel_array -= pixel_array.min()
                pixel_array /= pixel_array.max()
                pixel_array *= 255.0
                img = pixel_array.astype(np.uint8)

                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                cv2.imwrite(str(out_path), img)

            elif ext in ['.tif', '.tiff']:
                img = Image.open(f).convert("RGB")
                img.save(out_path)

            else:
                img = Image.open(f).convert("RGB")
                img.save(out_path)

            image_count += 1
            copied += 1

        except Exception as e:
            print(f"Skipped {f.name}: {e}")
            continue

    return copied


In [ ]:
full_train_general = './data/custom_dataset/norm/full_train/general'

print(convert_and_copy_images(general, full_train_general))
print(convert_and_copy_images(medical, full_train_general))
print(convert_and_copy_images(astronomy, full_train_general))
print(convert_and_copy_images(cctv, full_train_general))

In [10]:
from pathlib import Path

full_train_general = Path('./data/custom_dataset/norm/full_train/general')
file_count = len(list(full_train_general.glob('*')))  # or '*.jpg' if you want only images

print(f"Number of files: {file_count}")

Number of files: 32807


In [11]:
path_hr = './data/custom_dataset/norm/'
path_lr = './data/custom_dataset/bandw/'
pretrained_gen_path = './models/ColorizeArtistic_gen.pth'
proj_id = 'ArtisticModel'

gen_name = proj_id + '_gen'
pre_gen_name = gen_name + '_0'
crit_name = proj_id + '_crit'

name_gen = proj_id + '_image_gen'
path_gen = Path('./data/custom_dataset/name_gen')

TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

nf_factor = 1.5
pct_start = 1e-8

In [12]:
from fastai.vision import *
from sklearn.model_selection import train_test_split
from pathlib import Path

def get_data(bs:int, sz:int, keep_pct:float):
    return get_colorize_data(sz=sz, bs=bs, crappy_path=path_lr, good_path=path_hr,
                             random_seed=None, keep_pct=keep_pct)

def get_crit_data(train_files, valid_files, labels_train, labels_valid, bs, sz):
    all_files = train_files + valid_files
    all_labels = labels_train + labels_valid

    # Build label map BEFORE splitting
    label_map = {str(f): lbl for f, lbl in zip(all_files, all_labels)}

    # Create ImageList from all files
    item_list = ImageList(all_files)

    # Split first (important!)
    split_idx = list(range(len(train_files))), list(range(len(train_files), len(all_files)))
    split_items = item_list.split_by_idxs(*split_idx)

    # Then label using label_from_func (fastai v1 safe)
    labeled = split_items.label_from_func(lambda x: label_map[str(x)])

    # Final databunch
    data = (labeled
            .transform(get_transforms(max_zoom=2.), size=sz)
            .databunch(bs=bs)
            .normalize(imagenet_stats))

    return data

def save_preds(dl):
    i=0
    names = dl.dataset.items

    for b in dl:
        preds = learn_gen.pred_batch(batch=b, reconstruct=True)
        for o in preds:
            o.save(path_gen/names[i].name)
            i += 1

def save_gen_images():
    if path_gen.exists(): shutil.rmtree(path_gen)
    path_gen.mkdir(exist_ok=True)
    data_gen = get_data(bs=bs, sz=sz, keep_pct=0.085)
    save_preds(data_gen.fix_dl)
    PIL.Image.open(path_gen.ls()[0])

In [13]:
from pathlib import Path
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from skimage import exposure


def save_cv2(filename, image, subset):
    output_dir = Path(path_lr) / subset
    output_dir.mkdir(parents=True, exist_ok=True)
    dest = output_dir / filename
    cv2.imwrite(str(dest), image)

def save_cv2_before(filename, image, subset):
    output_dir = Path(path_hr) / subset
    output_dir.mkdir(parents=True, exist_ok=True)
    dest = output_dir / filename
    cv2.imwrite(str(dest), image)

def add_white_gaussian_noise(image_path, subset):
    image = cv2.imread(image_path)
    std = random.uniform(5, 40)  # camera ISO variability
    image_name = Path(image_path).stem
    filename = f"{image_name}_white_gaussian_noise.jpeg"
    save_cv2_before(filename, image, subset)
    noise = np.random.normal(0, std, image.shape).astype(np.float32)
    noisy = np.clip(image.astype(np.float32) + noise, 0, 255).astype(np.uint8)
    save_cv2(filename, noisy, subset)

def apply_gaussian_blur(image_path, subset):
    image = cv2.imread(image_path)
    ksize = random.choice([3, 5, 7])
    sigma = random.uniform(0.5, 2.5)
    image_name = Path(image_path).stem
    filename = f"{image_name}_gaussian_blur.jpeg"
    save_cv2_before(filename, image, subset)
    blurred = cv2.GaussianBlur(image, (ksize, ksize), sigmaX=sigma)
    save_cv2(filename, blurred, subset)

def jpeg_compression(image_path, subset):
    image = cv2.imread(image_path)
    quality = random.randint(5, 50)  # simulate variable lossy compression
    image_name = Path(image_path).stem
    filename = f"{image_name}_jpeg_compression.jpeg"
    save_cv2_before(filename, image, subset)
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, encimg = cv2.imencode('.jpg', image, encode_param)
    compressed = cv2.imdecode(encimg, 1)
    save_cv2(filename, compressed, subset)


def chromatic_aberration(image_path, subset):
    image = cv2.imread(image_path)
    shift = random.randint(1, 4)
    image_name = Path(image_path).stem
    filename = f"{image_name}_chromatic_aberration.jpeg"
    save_cv2_before(filename, image, subset)
    b, g, r = cv2.split(image)
    r = np.roll(r, shift, axis=1)
    b = np.roll(b, -shift, axis=1)
    aberrated = cv2.merge((b, g, r))
    save_cv2(filename, aberrated, subset)

def photometric_distortion(image_path, subset):
    image = cv2.imread(image_path)
    gamma = random.uniform(0.6, 1.4)
    image_name = Path(image_path).stem
    filename = f"{image_name}_photometric.jpeg"
    save_cv2_before(filename, image, subset)
    adjusted = exposure.adjust_gamma(image, gamma=gamma)
    adjusted = (adjusted * 255).astype(np.uint8)
    save_cv2(filename, adjusted, subset)

def structural_artifacts(image_path, subset):
    image = cv2.imread(image_path)
    spacing = random.randint(20, 50)
    thickness = random.randint(1, 3)
    image_name = Path(image_path).stem
    filename = f"{image_name}_structural_artifacts.jpeg"
    save_cv2_before(filename, image, subset)
    artifact = image.copy()
    h, w = artifact.shape[:2]
    for i in range(0, h, spacing):
        cv2.line(artifact, (0, i), (w, i), (255, 255, 255), thickness)
    save_cv2(filename, artifact, subset)

def apply_barrel_distortion(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_barrel_distortion.jpeg"
    save_cv2_before(filename, image, subset)
    h, w = image.shape[:2]
    k = random.uniform(0.00002, 0.0001)
    fx, fy = w, h
    cx, cy = w / 2, h / 2
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    D = np.array([-k, 0, 0, 0])
    map1, map2 = cv2.initUndistortRectifyMap(K, D, None, K, (w, h), cv2.CV_32FC1)
    distorted = cv2.remap(image, map1, map2, interpolation=cv2.INTER_LINEAR)
    save_cv2(filename, distorted, subset)

def apply_pincushion_distortion(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_pincushion_distortion.jpeg"
    save_cv2_before(filename, image, subset)
    h, w = image.shape[:2]
    k = random.uniform(0.00002, 0.0001)
    fx, fy = w, h
    cx, cy = w / 2, h / 2
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    D = np.array([k, 0, 0, 0])
    map1, map2 = cv2.initUndistortRectifyMap(K, D, None, K, (w, h), cv2.CV_32FC1)
    distorted = cv2.remap(image, map1, map2, interpolation=cv2.INTER_LINEAR)
    save_cv2(filename, distorted, subset)

def apply_mustache_distortion(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_mustache_distortion.jpeg"
    save_cv2_before(filename, image, subset)
    h, w = image.shape[:2]
    k1 = random.uniform(-0.0001, -0.00002)
    k2 = random.uniform(0.00002, 0.0001)
    fx, fy = w, h
    cx, cy = w / 2, h / 2
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    D = np.array([k1, k2, 0, 0])
    map1, map2 = cv2.initUndistortRectifyMap(K, D, None, K, (w, h), cv2.CV_32FC1)
    distorted = cv2.remap(image, map1, map2, interpolation=cv2.INTER_LINEAR)
    save_cv2(filename, distorted, subset)

# 1. Intensity Non-Uniformity (e.g., scanner illumination variation)
def intensity_non_uniformity(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_intensity_nonuniformity.jpeg"
    save_cv2_before(filename, image, subset)
    rows, cols = image.shape[:2]
    min_factor = random.uniform(0.4, 0.7)
    max_factor = random.uniform(1.2, 1.6)
    gradient = np.tile(np.linspace(min_factor, max_factor, cols), (rows, 1)).astype(np.float32)
    non_uniform = image.astype(np.float32)
    for i in range(3):
        non_uniform[:, :, i] *= gradient
    non_uniform = np.clip(non_uniform, 0, 255).astype(np.uint8)
    save_cv2(filename, non_uniform, subset)

# 2. Motion Artifacts (e.g., subject or sensor movement)
def motion_artifacts(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_motion_artifacts.jpeg"
    save_cv2_before(filename, image, subset)
    kernel_size = random.choice([7, 11, 15, 21])
    kernel_motion_blur = np.zeros((kernel_size, kernel_size))
    kernel_motion_blur[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
    kernel_motion_blur = kernel_motion_blur / kernel_size
    motion_blur = cv2.filter2D(image, -1, kernel_motion_blur)
    save_cv2(filename, motion_blur, subset)

# 3. Partial Volume Effects (e.g., pixel mixing at boundaries)
def partial_volume_effects(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_partial_volume.jpeg"
    save_cv2_before(filename, image, subset)
    scale = random.choice([1.5, 2.0, 2.5])
    down = cv2.resize(image, (int(image.shape[1] // scale), int(image.shape[0] // scale)))
    up = cv2.resize(down, (image.shape[1], image.shape[0]))
    alpha = random.uniform(0.3, 0.7)
    blended = cv2.addWeighted(image, alpha, up, 1 - alpha, 0)
    save_cv2(filename, blended, subset)

# 4. Beam Hardening (e.g., cupping effect in CT)
def beam_hardening(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_beam_hardening.jpeg"
    save_cv2_before(filename, image, subset)
    rows, cols = image.shape[:2]
    strength = random.uniform(0.3, 0.6)
    x = np.linspace(-1, 1, cols)
    y = np.linspace(-1, 1, rows)
    xv, yv = np.meshgrid(x, y)
    radius = np.sqrt(xv**2 + yv**2)
    mask = 1 - strength * radius
    mask = np.clip(mask, 0.5, 1.0)
    darkened = image.astype(np.float32)
    for i in range(3):
        darkened[:, :, i] *= mask
    darkened = np.clip(darkened, 0, 255).astype(np.uint8)
    save_cv2(filename, darkened, subset)

# 5. Metal Artifacts (e.g., bright streaks from implants in CT)
def metal_artifacts(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_metal_artifacts.jpeg"
    save_cv2_before(filename, image, subset)
    artifact = image.copy()
    h, w = artifact.shape[:2]
    spacing = random.randint(30, 80)
    thickness = random.randint(1, 3)
    for i in range(random.randint(10, 30), h, spacing):
        cv2.line(artifact, (0, i), (w, i), (255, 255, 255), thickness)
    for i in range(random.randint(10, 30), w, spacing):
        cv2.line(artifact, (i, 0), (i, h), (255, 255, 255), thickness)
    save_cv2(filename, artifact, subset)

# Compression Artifacts
def compression_artifacts(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    temp_file = f"{image_name}_compressed_temp.jpg"
    filename = f"{image_name}_compression_artifacts.jpeg"
    save_cv2_before(filename, image, subset)
    quality = random.randint(5, 15)  # very high compression
    cv2.imwrite(temp_file, image, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
    compressed = cv2.imread(temp_file)
    save_cv2(filename, compressed, subset)
    Path(temp_file).unlink()

# Low Resolution
def low_resolution(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_low_resolution.jpeg"
    save_cv2_before(filename, image, subset)
    factor = random.choice([4, 6, 8])
    small = cv2.resize(image, (image.shape[1]//factor, image.shape[0]//factor))
    low_res = cv2.resize(small, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
    save_cv2(filename, low_res, subset)

# Aliasing
def aliasing(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_aliasing.jpeg"
    save_cv2_before(filename, image, subset)
    rows, cols = image.shape[:2]
    freq = random.randint(2, 6)
    mask = np.indices((rows, cols)).sum(axis=0) % freq
    mask = cv2.resize((mask == 0).astype(np.uint8), (cols, rows), interpolation=cv2.INTER_NEAREST)
    aliased = image.copy()
    aliased[mask == 1] = 0
    save_cv2(filename, aliased, subset)

# Color Shift / White Balancing Error
def color_shift_white_balance(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_color_shift.jpeg"
    save_cv2_before(filename, image, subset)
    shift = np.array([
        random.randint(10, 30),
        random.randint(-20, 5),
        random.randint(-20, 5)
    ], dtype=np.int16)  # BGR shift
    shifted = image.astype(np.int16) + shift
    shifted = np.clip(shifted, 0, 255).astype(np.uint8)
    save_cv2(filename, shifted, subset)

# Rolling Shutter Distortion
def rolling_shutter_distortion(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_rolling_shutter.jpeg"
    save_cv2_before(filename, image, subset)
    distorted = image.copy()
    amplitude = random.randint(5, 15)
    frequency = random.uniform(80, 140)
    for i in range(image.shape[0]):
        offset = int(amplitude * np.sin(2 * np.pi * i / frequency))
        distorted[i] = np.roll(image[i], offset, axis=0)
    save_cv2(filename, distorted, subset)

# Atmospheric Distortion (Seeing)
def atmospheric_distortion(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_atmospheric_distortion.jpeg"
    save_cv2_before(filename, image, subset)
    for _ in range(random.randint(2, 5)):
        dx = np.random.normal(0, 1.5)
        dy = np.random.normal(0, 1.5)
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    blurred = cv2.GaussianBlur(image, (3, 3), sigmaX=1.2)
    save_cv2(filename, blurred, subset)


# Cosmic Ray Artifacts (bright pixels)
def cosmic_ray_artifacts(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_cosmic_ray.jpeg"
    save_cv2_before(filename, image, subset)
    result = image.copy()
    h, w = image.shape[:2]
    for _ in range(np.random.randint(5, 16)):
        x = np.random.randint(0, w)
        y = np.random.randint(0, h)
        radius = np.random.randint(1, 4)
        color = (255, 255, 255)
        cv2.circle(result, (x, y), radius, color, -1)
    save_cv2(filename, result, subset)

# Tracking Errors (drift or stacking misalignment)
def tracking_errors(image_path, subset):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    filename = f"{image_name}_tracking_error.jpeg"
    save_cv2_before(filename, image, subset)
    dx = random.randint(2, 6)
    dy = random.randint(2, 6)
    shifted1 = np.roll(image, dx, axis=1)
    shifted2 = np.roll(image, -dy, axis=0)
    stacked = cv2.addWeighted(image, 0.5, shifted1, 0.3, 0)
    stacked = cv2.addWeighted(stacked, 1.0, shifted2, 0.2, 0)
    save_cv2(filename, stacked, subset)


def create_general_training_images(fn, i, subset):
    add_white_gaussian_noise(fn, subset)
    apply_gaussian_blur(fn, subset)
    jpeg_compression(fn, subset)
    photometric_distortion(fn, subset)
    intensity_non_uniformity(fn, subset)
    motion_artifacts(fn, subset)
    partial_volume_effects(fn, subset)
    beam_hardening(fn, subset)
    metal_artifacts(fn, subset)
    compression_artifacts(fn, subset)
    low_resolution(fn, subset)
    aliasing(fn, subset)
    color_shift_white_balance(fn, subset)
    rolling_shutter_distortion(fn, subset)
    atmospheric_distortion(fn, subset)
    chromatic_aberration(fn, subset)
    cosmic_ray_artifacts(fn, subset)
    tracking_errors(fn, subset)


In [14]:
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import random


full_train_general = Path(full_train_general)
il = list(full_train_general.glob('*'))
random.shuffle(il)
print(len(il))

32807


In [15]:
# Split into 70% train, 15% val, 15% test
general_train_files, general_temp_files = train_test_split(il, test_size=0.3, random_state=42)
general_valid_files, general_test_files = train_test_split(general_temp_files, test_size=0.5, random_state=42)

# Launch for training set
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(create_general_training_images, str(fn), i, 'train') for i, fn in enumerate(general_train_files)]
    for f in futures:
        f.result()

# Launch for validation set
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(create_general_training_images, str(fn), i, 'valid') for i, fn in enumerate(general_valid_files)]
    for f in futures:
        f.result()

# Launch for test set
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(create_general_training_images, str(fn), i, 'test') for i, fn in enumerate(general_test_files)]
    for f in futures:
        f.result()


In [ ]:
bs=88
sz=64
keep_pct=1.0
data_gen = get_data(bs=bs, sz=sz, keep_pct=keep_pct)
learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor)
# state = torch.load(pretrained_gen_path, weights_only=False)
# learn_gen.model.load_state_dict(state['model'])
# learn_gen.callback_fns.append(partial(ImageGenTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GenPre'))
learn_gen.fit_one_cycle(1, pct_start=0.8, max_lr=slice(1e-4, 3e-4))
learn_gen.save(pre_gen_name)
learn_gen.unfreeze()
learn_gen.fit_one_cycle(1, pct_start=pct_start, max_lr=slice(1e-5, 1e-4))
learn_gen.save(pre_gen_name)

In [ ]:
bs=22
sz=128
keep_pct=1.0
learn_gen.data = get_data(sz=sz, bs=bs, keep_pct=keep_pct)
learn_gen.unfreeze()
learn_gen.fit_one_cycle(1, pct_start=pct_start, max_lr=slice(1e-7,1e-4))
learn_gen.save(pre_gen_name)

In [18]:
old_checkpoint_num = 0
checkpoint_num = old_checkpoint_num + 1
gen_old_checkpoint_name = gen_name + '_' + str(old_checkpoint_num)
gen_new_checkpoint_name = gen_name + '_' + str(checkpoint_num)
crit_old_checkpoint_name = crit_name + '_' + str(old_checkpoint_num)
crit_new_checkpoint_name= crit_name + '_' + str(checkpoint_num)

In [ ]:
bs=11
sz=128
data_gen_path = Path('./data/custom_dataset/bandw/models') / f'{gen_old_checkpoint_name}.pth'
learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor)
state = torch.load(Path(data_gen_path), weights_only=False)
learn_gen.model.load_state_dict(state['model'])
save_gen_images()

In [20]:
real_files = get_image_files(Path('./data/custom_dataset/norm/full_train'))
fake_files = get_image_files(Path('./data/custom_dataset/name_gen'))

real_labels = [1] * len(real_files)
fake_labels = [0] * len(fake_files)

all_files = real_files + fake_files
all_labels = real_labels + fake_labels

train_files, valid_files, labels_train, labels_valid = train_test_split(
    all_files, all_labels, test_size=0.2, stratify=all_labels, random_state=42
)

In [ ]:
if old_checkpoint_num == 0:
    bs=64
    sz=128
    learn_gen=None
    gc.collect()
    data_crit = get_crit_data(train_files, valid_files, labels_train, labels_valid, bs, sz)
    data_crit.show_batch(rows=3, ds_type=DatasetType.Train, imgsize=3)
    learn_critic = colorize_crit_learner(data=data_crit, nf=256)
    learn_critic.callback_fns.append(partial(LearnerTensorboardWriter, base_dir=TENSORBOARD_PATH, name='CriticPre'))
    learn_critic.fit_one_cycle(6, 1e-3)
    learn_critic.save(crit_old_checkpoint_name)

In [ ]:
bs=32
sz=128
data_crit = get_crit_data(train_files, valid_files, labels_train, labels_valid, bs, sz)
data_crit.show_batch(rows=3, ds_type=DatasetType.Train, imgsize=3)
learn_critic = colorize_crit_learner(data=data_crit, nf=256)
# state = torch.load(f'./models/{crit_old_checkpoint_name}.pth', weights_only=False)
# learn_critic.model.load_state_dict(state['model'])
learn_critic.callback_fns.append(partial(LearnerTensorboardWriter, base_dir=TENSORBOARD_PATH, name='CriticPre'))
learn_critic.fit_one_cycle(4, 1e-4)
learn_critic.save(crit_new_checkpoint_name)

In [ ]:
learn_crit=None
learn_gen=None
gc.collect()
lr=1e-5
sz=128
bs=12
data_crit = get_crit_data(train_files, valid_files, labels_train, labels_valid, bs, sz)
learn_crit = colorize_crit_learner(data=data_crit, nf=256)
state = torch.load(f'./models/{crit_new_checkpoint_name}.pth', weights_only=False)
learn_crit.model.load_state_dict(state['model'])

learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor)
#data/custom_dataset/bandw/models/ArtisticModel_gen_0.pth
state = torch.load(f'./data/custom_dataset/bandw/models/{gen_old_checkpoint_name}.pth', weights_only=False)
learn_gen.model.load_state_dict(state['model'])

switcher = partial(AdaptiveGANSwitcher, critic_thresh=0.65)
learn = GANLearner.from_learners(learn_gen, learn_crit, weights_gen=(1.0,2.0), show_img=False, switcher=switcher,
                                 opt_func=partial(optim.Adam, betas=(0.,0.9)), wd=1e-3)
learn.callback_fns.append(partial(GANDiscriminativeLR, mult_lr=5.))
learn.callback_fns.append(partial(GANTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GanLearner', visual_iters=100))
learn.callback_fns.append(partial(GANSaveCallback, learn_gen=learn_gen, filename=gen_new_checkpoint_name, save_iters=100))

In [ ]:
learn.data = get_data(sz=sz, bs=bs, keep_pct=0.03)
learn_gen.freeze_to(-1)
learn.fit(1,lr)

In [26]:
learn_gen.save(pre_gen_name)